In [1]:
import numpy as np
import pandas as pd
import json

root_joint_name = "m_avg_root"

# 获取映射表
mapping_table = pd.read_csv("keys_mapping_new.csv")
mapping_table.set_index('fbx_name', inplace=True)

# mapping_table

In [2]:
import os

def get_changed_extention_abs_path(abs_path,ext):
    return os.path.splitext(abs_path)[0]+"."+ext.replace(".","")

In [3]:

def json2amass_npz(json_path):

    pos_x = "position_x"
    pos_y = "position_y"
    pos_z = "position_z"
    rot_x = "rotation_x"
    rot_y = "rotation_y"
    rot_z = "rotation_z"

    with open(json_path,'r',encoding = 'utf-8') as fp:
        data = json.load(fp)
        frame_length = data["frame_length"]

        # 通用
        gender = 'male' # 性别
        mocap_framerate = 30 # 帧率
        betas = np.zeros(16) # 形状参数，默认为0
        dmpls = np.zeros((frame_length, 8)) # 软组织系数，默认为0

        # 更新
        trans = np.zeros((frame_length, 3)) # 全局平移
        poses = np.zeros((frame_length, 156)) # 姿势参数

        # 处理每帧数据
        for frame_data in data["frame_sequence"]:
            # 第几帧索引
            frame_index = frame_data["frame_number"]


            # 处理关节点数据
            for joint_data in frame_data["joint_position"]:
                
                joint_name = joint_data["joint_name"]

                if (joint_name == root_joint_name):
                            # 把pos赋给全局平移
                    trans[frame_index, :] = [joint_data[pos_x], 
                                            joint_data[pos_y], 
                                            joint_data[pos_z]]
                else:
                    # 尝试获取当前关节序号
                    try:
                        order = mapping_table.loc[joint_name]["order"]
                    except KeyError:
                        continue

                    poses[frame_index, order*3] = joint_data[rot_x]
                    poses[frame_index, order*3+1] = joint_data[rot_y]
                    poses[frame_index, order*3+2] = joint_data[rot_z]

        # 角度值转弧度制
        poses = np.deg2rad(poses)
        # print(trans)

        np.savez(get_changed_extention_abs_path(json_path,".npz"), trans=trans, gender=gender, mocap_framerate=mocap_framerate, betas=betas, dmpls=dmpls, poses=poses)


In [24]:
data_path = './TestData/'
file_name = 'a020_032100.json'


full_path = data_path + file_name

json2amass_npz(full_path)

In [30]:
import os
import glob

# 指定目录路径
dir_path = r'F:\AI Repository\Motion Diffusion\SoulsAnim\BB'

# 获取目录中所有的 JSON 文件
json_files = glob.glob(os.path.join(dir_path, '*.json'))

i = 0

# 输出 JSON 文件名列表
for file_path in json_files:
    # print(os.path.basename(file_path))
    json2amass_npz(file_path)

    i = i + 1
    print(i)
